In [1]:
#Importando as bibliotecas que serão necessárias
import pandas as pd # Manipulção de Dataframes 
import matplotlib.pyplot as plt # Manipulação e criação de gráficos
import seaborn as sns # Criação de gráficos

In [2]:
df = pd.read_csv(
    "Dataframes/houseprice.csv"
)
df.head()

,Id,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage,Price
0,1,1360,5,4,3,1970,Downtown,Excellent,No,149919
1,2,4272,5,4,3,1958,Downtown,Excellent,No,424998
2,3,3592,2,2,3,1938,Downtown,Good,No,266746
3,4,966,4,2,2,1902,Suburban,Fair,Yes,244020
4,5,4926,1,4,2,1975,Downtown,Fair,Yes,636056


In [3]:
print("Colunas originais: ",df.columns.tolist())

Colunas originais:  ['Id', 'Area', 'Bedrooms', 'Bathrooms', 'Floors', 'YearBuilt', 'Location', 'Condition', 'Garage', 'Price']


In [4]:
# Definição das variáveis que serão utilizadas (Features e Target)
# Target -> Variável que nos queremos prever ou classificar
# Features -> Variáveis que são correlacionadas com o Target
features = df[df.drop(['Id','Price'],axis=1).columns]
target = df['Price']
X = features
y = target

In [5]:
# Divisão de treino e teste dos dados
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2, # Distribuição dos dados (20%) para teste e (80%) para treino
    random_state=42   
)

In [34]:
X_train

,Area,Bedrooms,Bathrooms,Floors,YearBuilt,Location,Condition,Garage
968,4483,4,4,3,1933,Urban,Excellent,No
240,1062,3,3,1,1970,Downtown,Good,No
819,1422,3,4,1,1993,Urban,Good,Yes
692,2658,2,3,1,1972,Rural,Poor,Yes
420,3286,2,4,1,1981,Rural,Excellent,Yes
...,...,...,...,...,...,...,...,...
1130,3483,5,3,3,1931,Downtown,Good,Yes
1294,1747,3,3,2,1941,Downtown,Fair,Yes
860,1484,2,3,3,1973,Downtown,Good,No
1459,3456,5,4,1,1955,Downtown,Good,Yes


In [6]:
# Separação das colunas categóricas e númericas
colunas_categoricas = X_train.select_dtypes(include='object').columns.tolist()
colunas_numericas = X_train.select_dtypes(exclude='object').columns.tolist()

In [ ]:
# Criação do Pipeline de Pré-processamento de colunas (ColumnTransformer)
# Pipeline é uma sequência de processos 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        # Aplicando o escalonamento em colunas númericas, variáveis númericas em um intervalo de 0 e 1
        ('num', StandardScaler(),colunas_numericas),
        # Aplicando One-Hot Encoding em Colunas Categóricas 
        # handle_unknown='ignore' evita erros se uma nova categoria aparecer no teste
        ('cat', OneHotEncoder(handle_unknown='ignore',sparse_output=False), colunas_categoricas)
        # Passar o resto significa que qualquer coluna não especificada será descartada (o que é o padrão aqui)
    ],
    remainder='drop'
)

In [9]:
# Aplicando a transformação
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [11]:
# Número de features após o One-Hot Encoding
input_dim = X_train_processed.shape[1]
print(f"Número de Features após One-Hot e Escalonamento: {input_dim}")

Número de Features após One-Hot e Escalonamento: 15


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping
# Construção do Modelo MLP (Keras)
modelo = Sequential([
    # Camada de entrada / Oculta 1
    # 128 -> Número de neurônios
    # relu -> Camada oculta
    Dense(128, activation='relu', input_shape=(input_dim,)),
    # Camada Oculta 2
    Dense(64, activation='relu'),
    # Camada de saída
    # 1 Neurônio (previsão de um valor contínuo: o preço)
    # Ativação linear (padrão para regressão)
    Dense(1, activation='linear')
])

In [16]:
# Compilação
modelo.compile(
    optimizer='adam',
    # Perda padrão para Regressão
    loss='mean_squared_error',
    # Métrica de interpretação (em R$)
    metrics=[MeanAbsoluteError(name='MAE')] 
)

In [18]:
print("Estrutura do Modelo MLP:")
modelo.summary()

Estrutura do Modelo MLP:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,369 (40.50 KB)

 Trainable params: 10,369 (40.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Treinamento do modelo
# EarlyStopping para evitar Overfitting
es_callback = EarlyStopping(
    monitor='val_loss',  # Monitora a perda de validação
    patience=10,         # Espera 10 épocas sem melhora antes de parar
    restore_best_weights=True # Volta aos pesos da melhor época
)

In [22]:
# Treinamento
print("Iniciando Treinamento...")
history = modelo.fit(
    X_train_processed, 
    y_train,
    epochs=100,             # Número máximo de épocas (EarlyStopping para antes)
    batch_size=32,
    validation_split=0.15,  # 15% dos dados de treino para validação
    callbacks=[es_callback],
    verbose=0               # Não mostrar o log completo de cada época
)
print("Treinamento concluído. (Parou na época:", len(history.epoch), ")")

Iniciando Treinamento...
Treinamento concluído. (Parou na época: 69 )


In [24]:
# Avaliação e previsão
# Avaliação no Conjunto de Teste
loss, mae = modelo.evaluate(X_test_processed, y_test, verbose=0)

In [29]:
print("✅ Avaliação Final no Conjunto de Teste:")
print(f"Perda (MSE): {loss:,.0f}")
print(f"Erro Médio Absoluto (MAE): R$ {mae:,.2f}")

✅ Avaliação Final no Conjunto de Teste:
Perda (MSE): 78,891,507,712
Erro Médio Absoluto (MAE): R$ 245,055.64


In [33]:
# 6.2. Previsão para as 5 primeiras casas do teste
predictions = modelo.predict(X_test_processed[:5]).flatten()

print("\n📈 Previsões vs. Valores Reais:")
for i in range(5):
    preco_real = y_test.iloc[i]
    preco_previsto = predictions[i]
    diferenca = abs(preco_real - preco_previsto)
    
    print(f"Casa {i+1} | Real: R$ {preco_real:,.2f} | Previsto: R$ {preco_previsto:,.2f} | Diferença: R$ {diferenca:,.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

📈 Previsões vs. Valores Reais:
Casa 1 | Real: R$ 514,764.00 | Previsto: R$ 489,592.75 | Diferença: R$ 25,171.25
Casa 2 | Real: R$ 694,256.00 | Previsto: R$ 525,723.56 | Diferença: R$ 168,532.44
Casa 3 | Real: R$ 66,375.00 | Previsto: R$ 503,644.34 | Diferença: R$ 437,269.34
Casa 4 | Real: R$ 650,243.00 | Previsto: R$ 545,703.25 | Diferença: R$ 104,539.75
Casa 5 | Real: R$ 223,285.00 | Previsto: R$ 557,787.50 | Diferença: R$ 334,502.50


## 📝 Encerramento da Análise e Próximos Passos

Esta demonstração ilustrou com sucesso o processo completo de implementação do **Perceptron Multicamadas (MLP)** para um problema de regressão. Cobrimos todas as etapas essenciais: pré-processamento robusto (```StandardScaler``` e ```OneHotEncoder``` via ```ColumnTransformer```), definição da arquitetura de rede (saída linear e perda MSE), e o treinamento utilizando ```EarlyStopping```.

Apesar da correta aplicação do modelo, a avaliação final (como visto nas previsões) revelou uma **significativa disparidade** entre os valores preditos e os preços reais das casas. O **Erro Médio Absoluto (MAE)**, que representa o erro médio do nosso modelo em R$, indica que o MLP implementado, em sua configuração inicial, ainda não está apto para fazer previsões confiáveis.

---

## 🚀 Próximo Módulo de Análise

No próximo conjunto de análises, focaremos em **otimizar a precisão do modelo** e entender as causas da alta variância nos erros:

* **Análise de Dados Exploratória (EDA):** Iniciaremos com uma **análise detalhada das features.**

* **Matriz de Correlação:** Utilizaremos a **matriz de correlação** para quantificar a relação linear entre as features e a variável alvo (```Price```), identificando features fracas ou problemas de multicolinearidade.

* **Gráficos de Dispersão:** Gráficos de dispersão ajudarão a **visualizar a relação** das features chave (como ```Area``` e ```YearBuilt```) com o ```Price```, buscando identificar a presença de outliers ou de relações não-lineares fortes que o MLP deveria capturar.

* **Ajuste de Hiperparâmetros:** Faremos uma **nova rodada de treinamento** com hiperparâmetros ajustados (ex: reduzindo a taxa de aprendizado e ajustando o ```Dropout```) para buscar uma **convergência** mais estável e um MAE significativamente menor.